In [ ]:
import os
import openai
from openai import OpenAI
import requests

In [ ]:
# Set up your OpenAI API key
api_key = #placeholder
anthropic_api_key = #placeholder

# Define the path to the image file
image_path = '/Users/yumin/Desktop/998.bmp'
bbox_path = '/Users/yumin/Desktop/998_detections.json'
text_path = '/Users/yumin/Desktop/text/998.txt'
barcode_path = '/Users/yumin/Desktop/barcode/998.txt'

# prompt = "Please analyze this image and identify metadata elements such as text, figures, barcodes, and other details. Then, based on the detected metadata, generate the corresponding ZPL code. The text file includes coordinates for each element within the image. The format is [class] [x_center] [y_center] [width] [height], where 'class 0' indicates text, 'class 1' represents barcodes, and 'class 2' indicates lines or figures. "


# def get_zpl_code_from_image(image_path):
#     # Read the image file
#     with open(image_path, "rb") as image_file:
#         image_data = image_file.read()

#     # Upload the image and get the ZPL code by prompting ChatGPT with an appropriate request
#     response = openai.Image.create_edit(
#         image=image_data,
#         prompt=prompt,
#         size="1024x1024",
#         response_format="url"
#     )

#     if response:
#         print("ZPL Code Response:", response["choices"][0]["text"])
#     else:
#         print("Failed to retrieve the ZPL code.")

# # Call the function
# get_zpl_code_from_image(image_path)


# OPEN AI API

## Baseline

In [ ]:
from PIL import Image
import base64
import io
prompt = "please detect the metadata (e.g., text, figure, barcode, and other information) in this figure, and then generate the ZPL code based on the detected metadata"
def encode_image(image_path):
    # Open the BMP image
    with Image.open(image_path) as img:
        # Convert the image to PNG format in memory
        with io.BytesIO() as png_image:
            img.save(png_image, format="PNG")
            png_image.seek(0)  # Go back to the beginning of the in-memory file
            # Encode directly to base64
            return base64.b64encode(png_image.getvalue()).decode('utf-8')


# Path to your image
# image_path = "images/dogs/image_1.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

In [ ]:
from openai import OpenAI
import base64

client = OpenAI(api_key=api_key)
# client.api_key = api_key



response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"{prompt}"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                        # "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                    }
                },
            ],
        }
    ],
    max_tokens=3000,
)

print(response.choices[0].message.content)


To generate ZPL code for the content detected in the image, we first identify the text and barcode:

**Detected Metadata:**
1. Text:
   - "This format tests:"
   - "PDF417 (87), Mode: 0, CID with valid characters"

2. Barcode:
   - PDF417

**ZPL Code:**

```zpl
^XA
^FO50,50^ADN,36,20^FDThis format tests:^FS
^FO50,100^ADN,36,20^FDPDF417 (87), Mode: 0, CID with valid characters^FS
^FO50,150^B7N,8,10^FD1234567890^FS
^XZ
```

**Explanation:**
- `^XA` and `^XZ` are the start and end commands for a ZPL label.
- `^FO` sets the field origin, a position on the label.
- `^ADN,36,20` sets the font type and size.
- `^FD` specifies the field data to be printed.
- `^B7N,8,10` specifies a PDF417 barcode. The parameters define the barcode characteristics, such as height, security level etc.
- The `^FD1234567890^FS` in the barcode section is a placeholder for barcode data. Adjust it according to the actual data encoded in the image if needed.


## with Coordinates Information

In [ ]:
prompt = """please do the following steps to generate the ZPL code:
1/ detect the metadata in the uploaded image, including text, figure, barcode, and other information.
2/ Assign the coordinates to the detected metadata using the json_data. Do not use the "class". Only use the "bbox" for the coordinates of a metadata. The json file can contain less information than the bmp file, so do not rely on the json file.
3/ based on the detected metadata and bounding box for the metadata, generate the ZPL code.
"""


from PIL import Image
import base64
import io

def encode_image(image_path):
    # Open the BMP image
    with Image.open(image_path) as img:
        # Convert the image to PNG format in memory
        with io.BytesIO() as png_image:
            img.save(png_image, format="PNG")
            png_image.seek(0)  # Go back to the beginning of the in-memory file
            # Encode directly to base64
            return base64.b64encode(png_image.getvalue()).decode('utf-8')


# Path to your image
# image_path = "images/dogs/image_1.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

In [ ]:
from openai import OpenAI
import base64

client = OpenAI(api_key=api_key)
# client.api_key = api_key

import json
with open(bbox_path, 'r') as file:
    bbox_data = json.load(file)

# Convert the bbox_data to a JSON string
json_string = json.dumps(bbox_data)


response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"{prompt}json_data: f{json_string}"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                        # "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                    }
                },
                # {
                #     "type": "json_data",
                #     "json_data": bbox_json_string
                # }
            ],
        }
    ],
    max_tokens=3000,
)

print(response.choices[0].message.content)

To generate the ZPL code based on the image and JSON data provided, we'll perform the following steps:

### Detected Metadata
From the image, we detect:
- **Text**: "PDF417 (8?); Mode: 0, C10 with valid characters"
- **Barcode**: A barcode located as per the given JSON

### Assign Coordinates
Using the JSON data, the coordinates for the text and barcode are:

1. **Text**
   - Coordinates: `[15, 32, 294, 32]`
   - Start Point: `(15, 32)`
  
2. **Barcode**
   - Coordinates: `[15, 89, 196, 94]`
   - Start Point: `(15, 89)`

### Generate ZPL Code
Now, we use the detected data and their coordinates to generate the ZPL code.

```zpl
^XA
^FO15,32^A0N,30,30^FDThis format tests:^FS
^FO15,52^A0N,30,30^FDPDF417 (8?); Mode: 0, C10 with valid characters^FS
^FO15,89^BY2^B7N,3,10,,N^FD<The Barcode Data Here>^FS
^XZ
```

### Explanation
- `^XA`: Start the ZPL code.
- `^FOx,y`: Field Origin at coordinates (x, y).
- `^A0N,h,w`: Font selection with height (h) and width (w).
- `^FDtext^FS`: Field Data con

## with Coordinates Information and Text

In [ ]:
prompt = """please do the following steps to generate the ZPL code:
1/ detect the metadata in the uploaded image, including text, figure, barcode, and other information.
2/ make sure the text in the detected metadata is the same as the text_data. If they are not the same, use the text_data.
3/ Assign the coordinates to the detected metadata using the json_data. Do not use the "class". Only use the "bbox" for the coordinates of a metadata. The json file can contain less information than the bmp file, so do not rely on the json file.
4/ based on the detected metadata and bounding box for the metadata, generate the ZPL code.
"""

# Initialize a variable to store the detected text
detected_text = ""

# Open and read the file
with open(text_path, 'r') as file:
    for line in file:
        # Look for the line that contains "Detected Text:"
        if line.startswith("Detected Text:"):
            # Extract the text after "Detected Text:"
            detected_text = line.split("Detected Text:")[1].strip()
            break

# Print or use the extracted detected text
print("Detected Text:", detected_text)


from PIL import Image
import base64
import io

def encode_image(image_path):
    # Open the BMP image
    with Image.open(image_path) as img:
        # Convert the image to PNG format in memory
        with io.BytesIO() as png_image:
            img.save(png_image, format="PNG")
            png_image.seek(0)  # Go back to the beginning of the in-memory file
            # Encode directly to base64
            return base64.b64encode(png_image.getvalue()).decode('utf-8')


# Path to your image
# image_path = "images/dogs/image_1.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

Detected Text: PDF417 (8?); Mode: 0, C10 with valid characters


In [ ]:
from openai import OpenAI
import base64

client = OpenAI(api_key=api_key)
# client.api_key = api_key

import json
with open(bbox_path, 'r') as file:
    bbox_data = json.load(file)

# Convert the bbox_data to a JSON string
json_string = json.dumps(bbox_data)


response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"{prompt}\njson_data: f{json_string}\ntext_data: f{detected_text}"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                        # "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                    }
                },
                # {
                #     "type": "json_data",
                #     "json_data": bbox_json_string
                # }
            ],
        }
    ],
    max_tokens=3000,
    temperature=0,
)

print(response.choices[0].message.content)

To generate the ZPL code, follow these steps:

1. **Detect Metadata:**
   - From the image, identify the text and barcode.

2. **Verify Text:**
   - Ensure the detected text matches the `text_data`. If not, use the `text_data`.

3. **Assign Coordinates:**
   - Use the `bbox` from `json_data` to assign coordinates to the metadata.

4. **Generate ZPL Code:**

   - **Text:**
     - Coordinates from `json_data`: `[15, 32, 294, 32]`
     - Use `text_data`: `PDF417 (8?); Mode: 0, C10 with valid characters`

   - **Barcode:**
     - Coordinates from `json_data`: `[15, 89, 196, 94]`

Here is the ZPL code:

```zpl
^XA
^FO15,32^A0N,30,30^FDThis format tests:^FS
^FO15,62^A0N,30,30^FDPDF417 (8?); Mode: 0, C10 with valid characters^FS
^FO15,89^BY2
^B7N,5,200^FD1234567890^FS
^XZ
```

- `^XA` and `^XZ` are the start and end of the label format.
- `^FO` sets the field origin (x, y).
- `^A0N` sets the font and size.
- `^FD` defines the field data (text).
- `^BY` sets the barcode field default module wi

## with Coordinates Information, Text and Barcode

In [ ]:
prompt = """please do the following steps to generate the ZPL code:
1/ detect the metadata in the uploaded image, including text, figure, barcode, and other information.
2/ make sure the text in the detected metadata is the same as the text_data. If they are not the same, use the text_data.
3/ the number of barcode you should detect in the metadata is barcode_number.
4/ Assign the coordinates to the detected metadata using the json_data. Do not use the "class". Only use the "bbox" for the coordinates of a metadata. The json file can contain less information than the bmp file, so do not rely on the json file.
5/ based on the detected metadata and bounding box for the metadata, generate the ZPL code.
"""

# Initialize a variable to store the detected text
detected_text = ""

# Open and read the file
with open(text_path, 'r') as file:
    for line in file:
        # Look for the line that contains "Detected Text:"
        if line.startswith("Detected Text:"):
            # Extract the text after "Detected Text:"
            detected_text = line.split("Detected Text:")[1].strip()
            break

# Print or use the extracted detected text
print("Detected Text:", detected_text)

# Open and read the file
with open(barcode_path, 'r') as file:
    for line in file:
        # Look for the line that contains "Detected Text:"
        if line.startswith("Total Barcodes in Image:"):
            # Extract the text after "Detected Text:"
            detected_barcode = line.split("Total Barcodes in Image:")[1].strip()
            break

# Print or use the extracted detected text
print("Detected Barcode:", detected_barcode)




from PIL import Image
import base64
import io

def encode_image(image_path):
    # Open the BMP image
    with Image.open(image_path) as img:
        # Convert the image to PNG format in memory
        with io.BytesIO() as png_image:
            img.save(png_image, format="PNG")
            png_image.seek(0)  # Go back to the beginning of the in-memory file
            # Encode directly to base64
            return base64.b64encode(png_image.getvalue()).decode('utf-8')


# Path to your image
# image_path = "images/dogs/image_1.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

Detected Text: PDF417 (8?); Mode: 0, C10 with valid characters
Detected Barcode: 1


In [ ]:
from openai import OpenAI
import base64

client = OpenAI(api_key=api_key)
# client.api_key = api_key

import json
with open(bbox_path, 'r') as file:
    bbox_data = json.load(file)

# Convert the bbox_data to a JSON string
json_string = json.dumps(bbox_data)


response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": f"{prompt}\njson_data: f{json_string}\ntext_data: f{detected_text}\nbarcode_number: f{detected_barcode}"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                        # "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                    }
                },
                # {
                #     "type": "json_data",
                #     "json_data": bbox_json_string
                # }
            ],
        }
    ],
    max_tokens=3000,
    temperature=0,
)

print(response.choices[0].message.content)

To generate the ZPL code, follow these steps:

1. **Detect Metadata:**
   - The image contains text and a barcode.

2. **Verify Text:**
   - The detected text in the image is: "PDF417 (8?); Mode: 0, C10 with valid characters"
   - The `text_data` is: "PDF417 (8?); Mode: 0, C10 with valid characters"
   - The text matches, so no changes are needed.

3. **Barcode Detection:**
   - The image contains 1 barcode, which matches the `barcode_number`.

4. **Assign Coordinates:**
   - Use the `bbox` from `json_data` for coordinates:
     - Text: `[15, 32, 294, 32]`
     - Barcode: `[15, 89, 196, 94]`

5. **Generate ZPL Code:**

```zpl
^XA
^FO15,32^A0N,30,30^FDThis format tests:^FS
^FO15,64^A0N,30,30^FDPDF417 (8?); Mode: 0, C10 with valid characters^FS
^FO15,89^B7N,5,5,5,5,5^FD1234567890^FS
^XZ
```

- `^XA` and `^XZ` are the start and end of the ZPL code.
- `^FO` sets the field origin (x, y).
- `^A0N` sets the font and size for text.
- `^FD` specifies the data to print.
- `^B7N` is used for PDF4

# Anthropic

In [ ]:
import anthropic

client = anthropic.Anthropic(api_key=anthropic_api_key)

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type":"image/png",
                        "data": base64_image,
                    },
                },
                {
                    "type": "text",
                    "text": prompt
                }
            ],
        }
    ],
)
print(message.content[0].text)


From the image, I can detect:

1. Text header: "This format tests:"
2. Format specification: "PDF417 (3D), Mode: 0, C10 with valid characters"
3. A PDF417 2D barcode

To generate ZPL code for this, I would recommend:

```
^XA
^FO50,50
^A0N,30,30
^FDThis format tests:^FS
^FO50,90
^A0N,25,25
^FDPDF417 (3D), Mode: 0, C10 with valid characters^FS
^FO50,130
^BY3
^B7N,10,0,,,N
^FDYour_PDF417_Data^FS
^XZ
```

Where:
- ^XA starts the label format
- ^FO sets the field origin (x,y coordinates)
- ^A0N sets the font
- ^FD is the field data
- ^FS marks the field separator
- ^BY sets the bar code field defaults
- ^B7N specifies PDF417 barcode type
- ^XZ ends the label format

Note: The actual data content of the PDF417 barcode isn't visible in the image, so I've used "Your_PDF417_Data" as a placeholder. You would need to replace this with the actual data you want to encode in the barcode.
